# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 14 2023 8:52AM,257317,10,130639,"Virtue Rx, LLC",Released
1,Feb 14 2023 8:40AM,257313,10,0086378926,ISDIN Corporation,Released
2,Feb 14 2023 8:40AM,257313,10,0086378925,ISDIN Corporation,Released
3,Feb 14 2023 8:40AM,257313,10,0086378901,ISDIN Corporation,Released
4,Feb 14 2023 8:40AM,257313,10,0086378923,ISDIN Corporation,Released
5,Feb 14 2023 8:40AM,257313,10,0086378927,ISDIN Corporation,Released
6,Feb 14 2023 8:40AM,257313,10,0086378929,ISDIN Corporation,Released
7,Feb 14 2023 8:40AM,257313,10,0086378924,ISDIN Corporation,Released
8,Feb 14 2023 8:40AM,257313,10,0086378928,ISDIN Corporation,Released
9,Feb 14 2023 8:40AM,257313,10,0086378784,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,257313,Released,53
39,257314,Released,3
40,257315,Released,13
41,257316,Released,1
42,257317,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257313,NaN,53.0
257314,NaN,3.0
257315,NaN,13.0
257316,NaN,1.0
257317,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257203,0.0,20.0
257221,0.0,1.0
257231,0.0,63.0
257234,0.0,2.0
257236,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257203,0,20
257221,0,1
257231,0,63
257234,0,2
257236,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257203,0,20
1,257221,0,1
2,257231,0,63
3,257234,0,2
4,257236,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257203,,20
1,257221,,1
2,257231,,63
3,257234,,2
4,257236,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 14 2023 8:52AM,257317,10,"Virtue Rx, LLC"
1,Feb 14 2023 8:40AM,257313,10,ISDIN Corporation
54,Feb 14 2023 8:39AM,257312,10,ISDIN Corporation
79,Feb 14 2023 8:38AM,257316,12,Hush Hush
80,Feb 14 2023 8:37AM,257315,12,Hush Hush
93,Feb 14 2023 8:35AM,257314,10,"Methapharm, Inc."
95,Feb 14 2023 8:35AM,257314,10,Methapharm-G
96,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc."
102,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation
132,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 14 2023 8:52AM,257317,10,"Virtue Rx, LLC",,1
1,Feb 14 2023 8:40AM,257313,10,ISDIN Corporation,,53
2,Feb 14 2023 8:39AM,257312,10,ISDIN Corporation,,25
3,Feb 14 2023 8:38AM,257316,12,Hush Hush,,1
4,Feb 14 2023 8:37AM,257315,12,Hush Hush,,13
5,Feb 14 2023 8:35AM,257314,10,"Methapharm, Inc.",,3
6,Feb 14 2023 8:35AM,257314,10,Methapharm-G,,3
7,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc.",,6
8,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation,,33
9,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation,,50


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 8:52AM,257317,10,"Virtue Rx, LLC",1,
1,Feb 14 2023 8:40AM,257313,10,ISDIN Corporation,53,
2,Feb 14 2023 8:39AM,257312,10,ISDIN Corporation,25,
3,Feb 14 2023 8:38AM,257316,12,Hush Hush,1,
4,Feb 14 2023 8:37AM,257315,12,Hush Hush,13,
5,Feb 14 2023 8:35AM,257314,10,"Methapharm, Inc.",3,
6,Feb 14 2023 8:35AM,257314,10,Methapharm-G,3,
7,Feb 14 2023 8:30AM,257302,10,"Methapharm, Inc.",6,
8,Feb 14 2023 8:30AM,257301,10,ISDIN Corporation,33,
9,Feb 14 2023 8:30AM,257298,10,ISDIN Corporation,50,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 8:52AM,257317,10,"Virtue Rx, LLC",1,
1,Feb 14 2023 8:40AM,257313,10,ISDIN Corporation,53,
2,Feb 14 2023 8:39AM,257312,10,ISDIN Corporation,25,
3,Feb 14 2023 8:38AM,257316,12,Hush Hush,1,
4,Feb 14 2023 8:37AM,257315,12,Hush Hush,13,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 8:52AM,257317,10,"Virtue Rx, LLC",1.0,NaN
1,Feb 14 2023 8:40AM,257313,10,ISDIN Corporation,53.0,NaN
2,Feb 14 2023 8:39AM,257312,10,ISDIN Corporation,25.0,NaN
3,Feb 14 2023 8:38AM,257316,12,Hush Hush,1.0,NaN
4,Feb 14 2023 8:37AM,257315,12,Hush Hush,13.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 8:52AM,257317,10,"Virtue Rx, LLC",1.0,0.0
1,Feb 14 2023 8:40AM,257313,10,ISDIN Corporation,53.0,0.0
2,Feb 14 2023 8:39AM,257312,10,ISDIN Corporation,25.0,0.0
3,Feb 14 2023 8:38AM,257316,12,Hush Hush,1.0,0.0
4,Feb 14 2023 8:37AM,257315,12,Hush Hush,13.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6431953,314.0,2.0
12,771884,14.0,1.0
19,2315563,86.0,1.0
20,514598,4.0,0.0
21,771776,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6431953,314.0,2.0
1,12,771884,14.0,1.0
2,19,2315563,86.0,1.0
3,20,514598,4.0,0.0
4,21,771776,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,314.0,2.0
1,12,14.0,1.0
2,19,86.0,1.0
3,20,4.0,0.0
4,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,314.0
1,12,Released,14.0
2,19,Released,86.0
3,20,Released,4.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,21
Status,,,,,
Executing,2.0,1.0,1.0,0.0,0.0
Released,314.0,14.0,86.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,21
0,Executing,2.0,1.0,1.0,0.0,0.0
1,Released,314.0,14.0,86.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,21
0,Executing,2.0,1.0,1.0,0.0,0.0
1,Released,314.0,14.0,86.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()